In [1]:
from dendrocat import RadioSource
from astropy.io import fits
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import sys
import importlib
from astropy.wcs import WCS
from scipy import interpolate
import pandas as pd
import sympy as syp
from astropy import units as u
import radio_beam
from astropy.table import Table
from radio_beam import Beam
from astropy import coordinates
from astropy import wcs 
from astropy.nddata.utils import Cutout2D
from dendrocat.aperture import Ellipse
from regions import Regions, PixCoord
from astropy import stats
from itertools import chain
from radio_beam import Beams
from astropy.convolution import convolve
import astropy.constants as c



W51 = '/orange/adamginsburg/w51/TaehwaYoo/'
W51b6 = '/orange/adamginsburg/w51/TaehwaYoo/2015.1.01596.S_W51_B6_LB/continuum_images/'
W51cont='/orange/adamginsburg/w51/TaehwaYoo/b6contfits/'
w51e2_b6_briggs=W51cont+'W51e2_cont_bigbriggs.image.fits'
w51e2_b6_robust0=W51cont+'W51e2_cont_big_robust0.image.fits'
w51e2_b6_uniform=W51cont+'W51e2_cont_biguniform.image.fits'
w51e2_b6_superuniform=W51cont+'W51e2_cont_bigsuperuniform.image.fits'

w51n_b6_briggs = W51cont+'W51n_cont_bigbriggs.image.fits'
w51n_b6_robust0 = W51cont+'w51n_cont_big_robust0.image.fits'
w51n_b6_uniform = W51cont+'W51n_cont_biguniform.image.fits'
w51n_b6_superuniform = W51cont+'W51n_cont_bigsuperuniform.image.fits'
w51n_b6_natural = W51cont+'W51n_cont_bignatural.image.fits'

W51b3 = '/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/'

w51n_b3_tt0 = W51b3+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b3_tt1 = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51n_b3_alpha = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51conv = '/orange/adamginsburg/w51/TaehwaYoo/convolved_new/'
w51n_b6_conv = w51conv + 'w51n_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b3_tt0 = W51b3+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51e2_b3_tt1 = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51e2_b3_alpha = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51e2_b6_conv = w51conv + 'w51e2_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b6_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B6_conv.fits'
w51e_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B3_conv.fits'


w51n_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51n_B3_conv.fits'

w51n_b6_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'

w51n_b6_conv = w51conv + 'w51n_new_nocorr_in_area_B6_conv.fits'
#w51n_b3_conv = w51conv + 'w51n_B3_conv.fits'
#w51e_b3_conv = w51conv + 'w51e_B3_conv.fits'
w51e_b6_conv = w51conv + 'w51e_new_nocorr_in_area_B6_conv.fits'


w51e_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51e_matched.fits'
w51n_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51n_matched.fits'

w51e_b3_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51e_b3_std_sky.reg'
w51e_b6_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51e_b6_std_sky.reg'
w51n_b3_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51n_b3_std_sky.reg'
w51n_b6_noiseregion = '/orange/adamginsburg/w51/TaehwaYoo/w51n_b6_std_sky.reg'
w51e_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51n.spw0thru19.14500.robust0.thr0.1mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'


photometrydir = '/home/t.yoo/w51/catalogue/photometry/'
w51e_b3_flux = photometrydir+'w51e_b3_flux_size.fits'
w51n_b3_flux = photometrydir+'w51n_b3_flux_size.fits'
w51e_b6_flux = photometrydir+'w51e_b6_flux_size.fits'
w51n_b6_flux = photometrydir+'w51n_b6_flux_size.fits'
w51e_b6_conv_flux = photometrydir+'w51e_b6_conv_flux_size.fits'
w51n_b6_conv_flux = photometrydir+'w51n_b6_conv_flux_size.fits'







In [2]:
from astropy.table import Table
import matplotlib as mpl
import emcee
import corner
import scipy.integrate as integrate
from scipy.optimize import minimize, Bounds, LinearConstraint
#import os
#from multiprocessing import Pool

#os.environ["OMP_NUM_THREADS"] = "1"

def kappa(nu, nu0=271.1*u.GHz, kappa0=0.0114*u.cm**2*u.g**-1, beta=1.75):
    """
    Compute the opacity $\kappa$ given a reference frequency (or wavelength)
    and a power law governing the opacity as a fuction of frequency:
    $$ \kappa = \kappa_0 \left(\\frac{\\nu}{\\nu_0}\\right)^{\\beta} $$
    The default kappa=0.0114 at 271.1 GHz comes from extrapolating the
    Ossenkopf & Henning 1994 opacities for the thin-ice-mantle, 10^6 year model
    anchored at 1.0 mm with an assumed beta of 1.75.
    Parameters
    ----------
    nu: astropy.Quantity [u.spectral() equivalent]
        The frequency at which to evaluate kappa
    nu0: astropy.Quantity [u.spectral() equivalent]
        The reference frequency at which $\kappa$ is defined
    kappa0: astropy.Quantity [cm^2/g]
        The dust opacity per gram of H2 along the line of sight.  Because of
        the H2 conversion, this factor implicitly includes a dust to gas ratio
        (usually assumed 100)
    beta: float
        The power-law index governing kappa as a function of nu
    """
    return (kappa0*(nu.to(u.GHz,u.spectral())/nu0.to(u.GHz,u.spectral()))**(beta)).to(u.cm**2/u.g)

def find_opt_thick_rad(radarr, r, kappa, rho_0, r_0, alpha, rmax ):
    au_to_cm = (1*u.au).to(u.cm)
    find = False
    for rad in radarr:
        tau = kappa * rho_0 * r_0 * au_to_cm * r_0 **(alpha-1) * integrate.quad(lambda x: (np.sqrt(r**2+x**2))**(-alpha/2), rad, np.sqrt(rmax**2-r**2))[0]
        if tau.value > 1:
            rthick = rad
            find = True
            break
    tau_half = tau
    for rad in radarr:
        tau = tau_half + kappa * rho_0 * r_0 * au_to_cm * r_0 **(alpha-1) * integrate.quad(lambda x: (np.sqrt(r**2+x**2))**(-alpha/2), 0, rad)[0]
        if tau.value > 1:
            rthick = rad + np.max(radarr)
            find = True
            break
    
    if not find:
        rthick = -1
    return rthick, tau.value


def find_opt_thick_rad2(r, kappa, rho_0, r_0, alpha, rmax ):
    au_to_cm = (1*u.au).to(u.cm)
    
    radarr = np.linspace(np.sqrt(rmax**2-r**2),0, 500)
    radarr2 = np.linspace(0, np.sqrt(rmax**2-r**2), 500)

    dx = radarr[1]-radarr[0]
    y = (np.sqrt(r**2+radarr**2))**(-alpha)
    y2 = (np.sqrt(r**2+radarr2**2))**(-alpha)

    tau = kappa * rho_0 * r_0 * au_to_cm * r_0 **(alpha-1) *integrate.cumtrapz(y, radarr, dx=dx) * (-1)
    if np.max(tau.value)>1:
        ind = np.min(np.where(tau.value>1)[0])
        rthick = radarr[ind]
        
    else:
        tau_half = np.max(tau)
        tau = tau_half + kappa * rho_0 * r_0 * au_to_cm * r_0 **(alpha-1) *integrate.cumtrapz(y2, radarr2, dx=dx)
        if np.max(tau.value)>1:
            ind = np.min(np.where(tau.value>1)[0])
            rthick = radarr2[ind]+np.sqrt(rmax**2-r**2)
        else:
            rthick = -1
        
    return rthick, np.max(tau.value)


def BB(freq, temp):
    B_nu = (2 * freq**3 *c.h / (c.c**2) * 1 / (np.e**(c.h*freq/(c.k_B*temp))-1))
    return B_nu.to(u.Jy)
#def find_opt_thick_radius(rarr, kappa, rho):
#    
#    kappa*rho*
def tau(r, int_lolim, int_uplim, kappa, rho_0, alpha):
    return (kappa * rho_0 * integrate.quad(lambda x: (np.sqrt(r**2+x**2)/r_0.value)**(-alpha), int_lolim, int_uplim) *u.au).to(u.cm/u.cm)[0]  
def get_flux_aperture(theta, rarr, freqb3, freqb6, dist=5.41*u.kpc, r_0 = 100*u.au, verbose=False ):
    dr = rarr[1:] - rarr[:-1]
    dr = np.append(dr, rarr[-1])
    r_thick, rmax, logrho_0, alpha, T_0, beta = theta
    rho_0 = 10**logrho_0 *u.g / u.cm**3
    temp_0 = T_0 * u.K
    #print(rho_0)
    
    rho_r =  rho_0 * (rarr/r_0)**(-alpha)
    temp_r = temp_0 * (rarr/r_0)**(-beta)
    
    kappa_b3 = kappa(freqb3)
    kappa_b6 = kappa(freqb6)
    #print('ho',(rho_0*kappa_b3*r_0).to(u.cm/u.cm))

    au_to_cm = (1*u.au).to(u.cm)
    #print((kappa_b3*rho_0*r_0).to(u.au/u.au), alpha)
    #r_b3thick = (rmax**(1-alpha)-(1-alpha)/(kappa_b3.value*rho_0.value*(r_0.value*au_to_cm.value))*r_0.value**(1-alpha))**(1/(1-alpha))*u.au
    #r_b6thick = (rmax**(1-alpha)-(1-alpha)/(kappa_b6.value*rho_0.value*(r_0.value*au_to_cm.value))*r_0.value**(1-alpha))**(1/(1-alpha))*u.au
  

    #r_b3thick = (1-alpha + kappa_b3.value * rho_0.value * r_0.value * au_to_cm.value) / (kappa_b3.value * rho_0.value * (r_0.value * au_to_cm.value)**alpha)**(1/(1-alpha)) / au_to_cm.value
    #r_b6thick = (1-alpha + kappa_b6.value * rho_0.value * r_0.value * au_to_cm.value) / (kappa_b6.value * rho_0.value * (r_0.value * au_to_cm.value)**alpha)**(1/(1-alpha)) / au_to_cm.value
    #print('r_b3thick', r_b3thick)
    #print('r_b6thick', r_b6thick)
    
    #BBthick_b3 = BB(freqb3, T1*u.K)
    #BBthick_b6 = BB(freqb6, T1*u.K)
    
    #BBthin_b3 = BB(freqb3, T2*u.K)
    #BBthin_b6 = BB(freqb6, T2*u.K)
    
    flux_b3_arr = []
    flux_b6_arr =  []
    flux_b3 = 0*u.Jy
    flux_b6 = 0*u.Jy
    
    radarr = np.arange(1,500,step=1)
    for i,r in enumerate(rarr):
        if verbose:
            print('verbose on')
            print(i,r, r_thick)
        if r <r_thick:
            int_uplim = np.sqrt(rmax**2-r**2)
            int_lolim = np.sqrt(r_thick**2-r**2)
            tau_b3_max = tau(r, int_lolim, int_uplim, kappa_b3, rho_0, alpha)
            tau_b6_max = tau(r, int_lolim, int_uplim, kappa_b6, rho_0, alpha)

            integral_b3 = kappa_b3 * rho_0 * integrate.quad(lambda x: (np.sqrt(r**2+x**2)/r_0.value)**(-alpha)
                                                             *BB(freqb3, temp_0*(np.sqrt(r**2+x**2)/r_0.value)**(-beta))
                                                             *np.exp(-tau_b3_max+tau(r, int_lolim, np.sqrt(r**2+x**2), kappa_b3, rho_0, alpha)), 
                                                             int_lolim, int_uplim)[0]*u.au 
            integral_b6 = kappa_b6 * rho_0 * integrate.quad(lambda x: (np.sqrt(r**2+x**2)/r_0.value)**(-alpha)
                                                             *BB(freqb6, temp_0*(np.sqrt(r**2+x**2)/r_0.value)**(-beta))
                                                             *np.exp(-tau_b6_max+tau(r, int_lolim, np.sqrt(r**2+x**2), kappa_b6, rho_0, alpha)), 
                                                             int_lolim, int_uplim)[0]*u.au 
            I_b3 = BB(freqb3,  temp_0*(r_thick/r_0.value)**(-beta)) * np.exp(-tau_b3_max) + integral_b3
            I_b6 = BB(freqb6,  temp_0*(r_thick/r_0.value)**(-beta)) * np.exp(-tau_b6_max) + integral_b6

            
        elif r < r_max:
            int_uplim = np.sqrt(rmax**2-r**2)
            int_lolim = -int_uplim
            tau_b3_max = 2*tau(r, int_lolim, int_uplim, kappa_b3, rho_0, alpha)
            tau_b6_max = 2*tau(r, int_lolim, int_uplim, kappa_b6, rho_0, alpha)

            integral_b3 = kappa_b3 * rho_0 * integrate.quad(lambda x: (np.sqrt(r**2+x**2)/r_0.value)**(-alpha)
                                                             *BB(freqb3, temp_0*(np.sqrt(r**2+x**2)/r_0.value)**(-beta))
                                                             *np.exp(-tau_b3_max+tau(r, int_lolim, np.sqrt(r**2+x**2), kappa_b3, rho_0, alpha)), 
                                                             int_lolim, int_uplim)[0] 
            integral_b6 = kappa_b6 * rho_0 * integrate.quad(lambda x: (np.sqrt(r**2+x**2)/r_0.value)**(-alpha)
                                                             *BB(freqb6, temp_0*(np.sqrt(r**2+x**2)/r_0.value)**(-beta))
                                                             *np.exp(-tau_b6_max+tau(r, int_lolim, np.sqrt(r**2+x**2), kappa_b6, rho_0, alpha)), 
                                                             int_lolim, int_uplim)[0] 
            I_b3 = integral_b3
            I_b6 = integral_b6
       
            
        else:
            I_b3=0
            I_b6=0
             
        flux_b3 = flux_b3 + 2 * np.pi * r*u.au * dr[i]*u.au * I_b3 /dist**2
        flux_b6 = flux_b6 + 2 * np.pi * r*u.au * dr[i]*u.au * I_b6 /dist**2
        if verbose:
            print('r, r_thick, flux_b3, flux_b6, I_b3, I_b6, tau_b3, tau_b6, rho_0, kappa_b3, kappa_b6')
            print(r, r_thick, flux_b3, flux_b6, I_b3, I_b6, tau_b3, tau_b6, rho_0, kappa_b3, kappa_b6) 
        if any((~np.isfinite(flux_b3),~np.isfinite(flux_b6))):
            print('r,r_thick, flux_b3, flux_b6, I_b3, I_b6, tau_b3, tau_b6, rho_0, kappa_b3, kappa_b6')
            print(r, r_thick, flux_b3, flux_b6, I_b3, I_b6, tau_b3, tau_b6, rho_0, kappa_b3, kappa_b6) 
            raise ValueError('nan in flux calc')
        #print('flux_b3',flux_b3)

        flux_b3_arr.append(flux_b3.to(u.Jy).value)
        flux_b6_arr.append(flux_b6.to(u.Jy).value)
        
        
    
    return flux_b3_arr, flux_b6_arr



def lnlike(theta, flux_b3, flux_b6, fluxerr_b3, fluxerr_b6, rarr,
           dist=5.41*u.kpc, freqb3=92982346121.91989*u.Hz, freqb6=226691598706.70853*u.Hz):
    model_b3, model_b6 = get_flux_aperture(theta, rarr, freqb3, freqb6, dist=dist)
    #print('hoho', flux_b3,model_b3)
    fluxerr_b3 = fluxerr_b3 * flux_b3/np.max(flux_b3)
    fluxerr_b6 = fluxerr_b6 * flux_b6/np.max(flux_b6)

    
    return -0.5* (np.sum((flux_b3-model_b3)**2/fluxerr_b3**2) + 
                  np.sum((flux_b6-model_b6)**2/fluxerr_b6**2))


def lnprior(theta):
    r_thick, rmax, logrho_0, alpha, T_0, beta= theta
    if 0<r_thick<rmax and r_thick<rmax<500 and -18 < logrho_0 < -4 and 0<alpha<5 and 0<T_0<500 and 0<beta<2:
        return 0.0
    return -np.inf

def lnprob(theta, flux_b3, flux_b6, fluxerr_b3, fluxerr_b6, rarr):
    lp = lnprior(theta) 
    if not np.isfinite(lp):
        return - np.inf
    return lp + lnlike(theta, flux_b3, flux_b6, fluxerr_b3, fluxerr_b6, rarr)

def main(p0,nwalkers,niter,ndim,lnprob,data):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=data)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter)

    return sampler, pos, prob, state


def flux_gaussian_multiple_apertures(flux, major, minor, rarr): # major, minor in FWHM, au unit
    flux_rescaled = flux*major/minor # making a 2d symmetric Gaussian with FWHM = FWHM_major
    major_sigma = major/np.sqrt(8*np.log(2))
    peak_height = flux / 2 / np.pi/ major/ minor * 8 * np.log(2)
    flux_rarr =2*np.pi*major_sigma**2 * peak_height * (1 - np.exp(-rarr**2/2/major_sigma**2))
    return flux_rarr
def get_mass(rho_0, r_0, alpha, rthick):
    mass = rho_0 * u.g /u.cm**3 * 4* np.pi /(3-alpha) * ((500/r_0)**(3-alpha) - (rthick/r_0)**(3-alpha)) * u.au **3
    return mass.to(u.Msun)
def get_fluxmass(freq, kappa, dist, peak_flux, int_flux, temp):
    mass = - 1 / kappa * dist**2 * int_flux / peak_flux * np.log(1 - (peak_flux  / BB(freq,temp)).to(u.cm/u.cm).value)
    return mass.to(u.Msun)

def get_fluxmass2(freq, kappa, dist, flux, temp):
    mass = flux * dist**2 /kappa/BB(freq,temp)
    return mass.to(u.Msun)


### W51-E

In [3]:
import time
w51e_b3_flux_fits = Table.read(w51e_b3_flux)
w51e_b6_flux_fits = Table.read(w51e_b6_flux)
w51n_b3_flux_fits = Table.read(w51n_b3_flux)
w51n_b6_flux_fits = Table.read(w51n_b6_flux)
w51e_b6_flux_conv_fits = Table.read(w51e_b6_conv_flux)
w51n_b6_flux_conv_fits = Table.read(w51n_b6_conv_flux)

w51e_b3_major = w51e_b3_flux_fits['deconvolved_major'].value* np.pi/180 * 5.41*1000*206265
w51e_b6_major = w51e_b6_flux_fits['deconvolved_major'].value* np.pi/180 * 5.41*1000*206265
w51n_b3_major = w51n_b3_flux_fits['deconvolved_major'].value* np.pi/180 * 5.1*1000*206265
w51n_b6_major = w51n_b6_flux_fits['deconvolved_major'].value* np.pi/180 * 5.1*1000*206265
w51e_b3_minor = w51e_b3_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.41*1000*206265
w51e_b6_minor = w51e_b6_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.41*1000*206265
w51n_b3_minor = w51n_b3_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.1*1000*206265
w51n_b6_minor = w51n_b6_flux_fits['deconvolved_minor'].value* np.pi/180 * 5.1*1000*206265



w51e_b3_int_flux = w51e_b3_flux_fits['flux']
w51e_b6_int_flux = w51e_b6_flux_conv_fits['flux']
w51n_b3_int_flux = w51n_b3_flux_fits['flux']
w51n_b6_int_flux = w51n_b6_flux_conv_fits['flux']

w51e_b3_int_fluxerr = w51e_b3_flux_fits['flux_upperr'] - w51e_b3_flux_fits['flux_lowerr']
w51e_b6_int_fluxerr = w51e_b6_flux_conv_fits['flux_upperr'] - w51e_b6_flux_conv_fits['flux_lowerr']
w51n_b3_int_fluxerr = w51n_b3_flux_fits['flux_upperr'] - w51n_b3_flux_fits['flux_lowerr']
w51n_b6_int_fluxerr = w51n_b6_flux_conv_fits['flux_upperr'] - w51n_b6_flux_conv_fits['flux_lowerr']
rmax=500
num=15
rarr = np.logspace(0,np.log10(rmax),num=num)
labels = [r'$R_{\rm thick}$ (AU)', r'$R_{\rm max}$ (AU)', r'log $\rho_0$ $(g/cm^3)$',r'$\alpha$','$T_0$ (K)',r'$\beta$']
freqb3=92982346121.91989*u.Hz
freqb6=226691598706.70853*u.Hz
kappa_b3 = kappa(freqb3)
kappa_b6 = kappa(freqb6)
au_to_cm = (1*u.au).to(u.cm)
g_to_msun = (1*u.g).to(u.Msun)
dist = 5.41*u.kpc
r_0 = 100*u.au


rthick_max = []
rthick_med = []
rthick_upp = []
rthick_low = []
t1_max = []
t1_med = []
t1_upp = []
t1_low = []
t2_max = []
t2_med = []
t2_upp = []
t2_low = []
rho0_max = []
rho0_med = []
rho0_upp = []
rho0_low = []
alpha_max = []
alpha_med = []
alpha_upp = []
alpha_low = []

nwalkers=100
niter=1000

for i in range(len(w51e_b3_int_flux)):
    if any((w51e_b3_int_flux[i]<0, w51e_b6_int_flux[i]<0,
            ~np.isfinite(w51e_b3_int_flux[i]), ~np.isfinite(w51e_b6_int_flux[i]), 
            w51e_b3_major[i]<=0, w51e_b6_major[i]<=0)):
        continue
    start=time.time()
   
    
    
    flux_b3 = flux_gaussian_multiple_apertures(w51e_b3_int_flux[i],w51e_b3_major[i], w51e_b3_minor[i], rarr)
    flux_b6 = flux_gaussian_multiple_apertures(w51e_b6_int_flux[i],w51e_b6_major[i], w51e_b6_minor[i], rarr)
    
    
    spectral_index = np.log(flux_b3[-1]/flux_b6[-1])/np.log(freqb3/freqb6)
    print('flux_apertures', flux_b3, flux_b6)
    print('spectral index',spectral_index )
    print('flux', w51e_b3_int_flux[i], w51e_b6_int_flux[i])
    if any((spectral_index>4, spectral_index<1.7)):
        rthick_max.append(-9999)
        rthick_med.append(-9999)
        rthick_upp.append(-9999)
        rthick_low.append(-9999)
        t1_max.append(-9999)
        t1_med.append(-9999)
        t1_upp.append(-9999)
        t1_low.append(-9999)
        t2_max.append(-9999)
        t2_med.append(-9999)
        t2_upp.append(-9999)
        t2_low.append(-9999)
        rho0_max.append(-9999)
        rho0_med.append(-9999)
        rho0_upp.append(-9999)
        rho0_low.append(-9999)
        alpha_max.append(-9999)
        alpha_med.append(-9999)
        alpha_upp.append(-9999)
        alpha_low.append(-9999)

        continue
        
        
    fluxerr_b3 = w51e_b3_int_fluxerr[i] * flux_b3/np.max(flux_b3)
    fluxerr_b6 = w51e_b6_int_fluxerr[i] * flux_b6/np.max(flux_b6)
    
    nll = lambda *args: -lnlike(*args)
    initial = np.array([100, 250, -14 ,2, 50, 0.5])
    ndim = len(initial)
    p0 = [np.array(initial) + [20,20,2,0.1, 20,0.1] * np.random.randn(ndim) for i in range(nwalkers)]
    data = (flux_b3, flux_b6, w51e_b3_int_fluxerr[i], w51e_b6_int_fluxerr[i], rarr)
    initial_var = np.array(initial) + [20,20,2,0.1, 20,0.1] * np.random.randn(ndim)
    bounds = Bounds([0,0,-18,0,0,0],[500,500,-4,5,500,2])
    constraint = {'type': 'ineq',
                 'fun': lambda x: np.array(x[0]-x[1],
                                         )}
    
    for k in range(500):
        soln = minimize(nll, initial_var, args=data, method='SLSQP', bounds=bounds,constraints=constraint)
        diff = (initial_var-soln.x)/initial_var
        initial_var = soln.x
        if np.max(diff)<1e-2:
            break

    initial = initial_var
    print('optimization',initial)
    p0 = [np.array(initial) + 0.01*np.array(initial) * np.random.randn(ndim) for i in range(nwalkers)]

    sampler, pos, prob, state = main(p0,nwalkers,niter,ndim,lnprob,data)
    print('elapsed time, ',time.time()-start)

    samples_flat = sampler.flatchain
    print(samples_flat[np.argmax(sampler.flatlnprobability)])
    theta_max  = samples_flat[np.argmax(sampler.flatlnprobability)]
    alpha = theta_max[4]
    rho_0 = 10**theta_max[3]
   
    mass = get_mass(10**theta_max[3],r_0.value, theta_max[4], theta_max[0])
    opt_thick_mass = rho_0 * (theta_max[0]/r_0.value)**(-alpha) * (4* np.pi /3) * theta_max[0]**3 * au_to_cm.value **3 * g_to_msun.value

    temp_massweighted = (theta_max[1] * opt_thick_mass + theta_max[2] * mass.value) / (opt_thick_mass + mass.value)

    fluxmass_b3 = get_fluxmass2(freqb3, kappa_b3, dist, flux_b3[-1]*u.Jy, temp_massweighted*u.K)
    fluxmass_b6 = get_fluxmass2(freqb6, kappa_b6, dist, flux_b6[-1]*u.Jy, temp_massweighted*u.K)

    
    f1,f2 = get_flux_aperture(theta_max, rarr, freqb3, freqb6, verbose=True )

    fig = corner.corner(samples_flat,show_titles=True,labels=labels,plot_datapoints=True,quantiles=[0.16, 0.5, 0.84])
    plt.savefig('mcmc_image/corner_%03d.png'%i)
    plt.show()
    plt.close()
    fig = plt.figure(figsize=(8,8))
    model_b3, model_b6 = get_flux_aperture(theta_max, rarr, freqb3, freqb6, dist=dist )
    
    plt.plot(rarr, model_b3, c='orange', label='model_b3')
    plt.plot(rarr, model_b6, c='cyan', label='model_b6')
    plt.errorbar(rarr, flux_b3, yerr=np.ones(len(flux_b3))*w51e_b3_int_fluxerr[i]*flux_b3/np.max(flux_b3), fmt='o', c='r', label='obs_b3')
    plt.errorbar(rarr, flux_b6, yerr=np.ones(len(flux_b6))*w51e_b6_int_fluxerr[i]*flux_b6/np.max(flux_b6), fmt='o', c='b', label='obs_b6')
    #plt.text(350, np.max(flux_b6)*0.6, 'r_b6thick = %3.2e'%r_b6thick.value+' au')
    #plt.text(350, np.max(flux_b6)*0.5, 'r_b3thick = %3.2e'%r_b3thick.value+' au')
    plt.text(200, np.max(flux_b6)*0.7, 'mass_weighted temp = %3.2e K'%temp_massweighted)
    plt.text(200, np.max(flux_b6)*0.3, 'envelope mass = %3.2e Msun'%(mass.value))
    plt.text(200, np.max(flux_b6)*0.4, 'opt_thick_mass = %3.2e Msun'%(opt_thick_mass))
    plt.text(200, np.max(flux_b6)*0.5, 'mass with flux+temp (B3) = %3.2e Msun'%(fluxmass_b3.value))
    plt.text(200, np.max(flux_b6)*0.6, 'mass with flux+temp (B6) = %3.2e Msun'%(fluxmass_b6.value))


    plt.legend()
    plt.xlabel('r (au)')
    plt.ylabel('flux (Jy)')
    #plt.xscale('log')
    #plt.yscale('log')
    plt.savefig('mcmc_image/radial_profile_%03d.png'%i)
    plt.show()
    plt.close()
    
    rthick_max.append(theta_max[0])
    t1_max.append(theta_max[1])
    t2_max.append(theta_max[2])
    rho0_max.append(theta_max[3])
    alpha_max.append(theta_max[4])
    
    a,b,c = np.percentiles(samples_flat[:,0], [16,50,84])
    rthick_low.append(a) ; rthick_med.append(b) ; rthick_upp.append(c)
    a,b,c = np.percentiles(samples_flat[:,1], [16,50,84])
    t1_low.append(a) ; t1_med.append(b) ; t1_upp.append(c)
    a,b,c = np.percentiles(samples_flat[:,2], [16,50,84])
    t2_low.append(a) ; t2_med.append(b) ; t2_upp.append(c)
    a,b,c = np.percentiles(samples_flat[:,3], [16,50,84])
    rho0_low.append(a) ; rho0_med.append(b) ; rho0_upp.append(c)
    a,b,c = np.percentiles(samples_flat[:,4], [16,50,84])
    alpha_low.append(a) ; alpha_med.append(b) ; alpha_upp.append(c)
        
table_write = Table([rthick_max, rthick_upp, rthick_med, rthick_low, 
                    t1_max, t1_upp, t1_med, t1_low,
                    t2_max, t2_upp, t2_med, t2_low,
                    rho0_max, rho0_upp, rho0_med, rho0_low,
                    alpha_max, alpha_upp, alpha_med, alpha_low], 
                    names=('rthick_max', 'rthick_upp', 'rthick_med', 'rthick_low', 
                    't1_max', 't1_upp', 't1_med', 't1_low',
                    't2_max', 't2_upp', 't2_med', 't2_low',
                    'logrho0_max', 'logrho0_upp', 'logrho0_med', 'logrho0_low',
                    'alpha_max', 'alpha_upp', 'alpha_med', 'alpha_low'))



table_write.write('mcmc_w51e.fits',format='fits',overwrite=True)
    

flux_apertures [3.78022110e-08 9.18476656e-08 2.23149465e-07 5.42084793e-07
 1.31644191e-06 3.19450581e-06 7.73746387e-06 1.86568228e-05
 4.44993780e-05 1.03411665e-04 2.26129001e-04 4.32443179e-04
 6.44318509e-04 7.20128682e-04 7.23473378e-04] [1.05276899e-06 2.55773822e-06 6.21320958e-06 1.50876857e-05
 3.66063973e-05 8.86314570e-05 2.13515725e-04 5.08148152e-04
 1.17472712e-03 2.53819175e-03 4.73448861e-03 6.78422727e-03
 7.38795290e-03 7.40594549e-03 7.40594878e-03]
spectral index 2.6099922348857896
flux 0.0006115890066178156 0.006805512272383893


TypeError: only dimensionless scalar quantities can be converted to Python scalars